In [1]:
pip install audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 1.6 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [5]:
# Define the path to the directory containing the spectrogram images
spectrogram_dir = '/content/drive/MyDrive/music genre classification/final/New_spectrograms'

# Define the model hyperparameters
input_shape = (64, 64, 3)  # Adjust the dimensions as per your spectrogram size
num_classes = len(os.listdir(spectrogram_dir))  # Assuming subdirectories represent genres


# Load spectrogram images and labels
spectrograms = []
labels = []

for genre_folder in os.listdir(spectrogram_dir):
    genre_dir = os.path.join(spectrogram_dir, genre_folder)
    for filename in os.listdir(genre_dir):
        if filename.endswith('.png'):
            filepath = os.path.join(genre_dir, filename)
            spectrogram = tf.keras.preprocessing.image.load_img(filepath, target_size=(input_shape[0], input_shape[1]))
            spectrogram = tf.keras.preprocessing.image.img_to_array(spectrogram)
            spectrogram /= 255.0  # Normalize pixel values to [0, 1]
            spectrograms.append(spectrogram)
            labels.append(genre_folder)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Sort the data by labels
sorting_indices = np.argsort(labels_encoded)
spectrograms = np.array(spectrograms)[sorting_indices]
labels_encoded = labels_encoded[sorting_indices]

num_classes = len(label_encoder.classes_)
labels_one_hot = to_categorical(labels_encoded, num_classes=num_classes)

# Split the data into training and testing sets
X_train_spectrogram, X_test_spectrogram, y_train_spectrogram, y_test_spectrogram = train_test_split(spectrograms, labels_one_hot, test_size=0.2, random_state=42)

joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [6]:
data = pd.read_csv('/content/drive/MyDrive/music genre classification/final/csv_files/gtzan_augmented_features_with_segments_20_mfccs.csv')

X = data.drop(['filename', 'label'], axis=1)
y = data['label']
X = X.values
y = y.values

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder=LabelEncoder()
y=to_categorical(label_encoder.fit_transform(y))


X_train_feature, X_test_feature, y_train_feature, y_test_feature = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_feature = scaler.fit_transform(X_train_feature)
X_test_feature = scaler.transform(X_test_feature)

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [7]:
y_train_feature.shape

(31971, 10)

In [8]:
y_train_spectrogram.shape

(31971, 10)

In [9]:
y_test_feature

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [10]:
y_test_spectrogram

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
X_test_feature = X_test_ann
y_test_feature = y_test_ann

X_test_spectrogram = X_test_cnn
y_test_spectrogram = y_test_cnn

In [11]:
from tensorflow.keras.models import load_model

feature_model1 = load_model('/content/ann_20_mfccs_model.h5')  # Replace with the actual path to your model
feature_model2 = load_model('/content/cnn_20_mfccs_model.h5')  # Replace with the actual path to your model

spectrogram_model1 = load_model('/content/CNN_model_MGC_1.h5')  # Replace with the actual path to your model
spectrogram_model2 = load_model('/content/GRU_model_MGC_1.h5')  # Replace with the actual path to your model
spectrogram_model3 = load_model('/content/LSTM_model_MGC_1.h5')

# Define a list of corrected feature models
feature_models = [feature_model1, feature_model2]

# Define a list of corrected spectrogram models
spectrogram_models = [spectrogram_model1, spectrogram_model2, spectrogram_model3]



# Function to test ensemble model
def test_ensemble_model(X_test_feature, X_test_spectrogram, y_test_feature, y_test_spectrogram,
                        feature_models, spectrogram_models):
    # Predict using feature models
    feature_preds = np.mean([model.predict(X_test_feature) for model in feature_models], axis=0)
    feature_preds_classes = np.argmax(feature_preds, axis=1)
    feature_acc = accuracy_score(np.argmax(y_test_feature, axis=1), feature_preds_classes)

    # Predict using spectrogram models
    spectrogram_preds = np.mean([model.predict(X_test_spectrogram) for model in spectrogram_models], axis=0)
    spectrogram_preds_classes = np.argmax(spectrogram_preds, axis=1)
    spectrogram_acc = accuracy_score(np.argmax(y_test_spectrogram, axis=1), spectrogram_preds_classes)

    # Combine the results from feature and spectrogram models
    ensemble_preds = np.mean([feature_preds, spectrogram_preds], axis=0)
    ensemble_preds_classes = np.argmax(ensemble_preds, axis=1)
    ensemble_acc = accuracy_score(np.argmax(y_test_feature, axis=1), ensemble_preds_classes)

    print(f'Feature Models Accuracy: {feature_acc * 100:.2f}%')
    print(f'Spectrogram Models Accuracy: {spectrogram_acc * 100:.2f}%')
    print(f'Ensemble Model Accuracy: {ensemble_acc * 100:.2f}%')

# Example usage:
test_ensemble_model(X_test_feature, X_test_spectrogram, y_test_feature, y_test_spectrogram,
                    feature_models, spectrogram_models)

250/250 [==============================] - 16s 60ms/step
Feature Models Accuracy: 95.17%
Spectrogram Models Accuracy: 96.22%
Ensemble Model Accuracy: 99.27%


Feature Models Accuracy: 94.88%
Spectrogram Models Accuracy: 96.17%
Ensemble Model Accuracy: 99.21%

In [12]:
import os
import tempfile
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from collections import Counter
tf.keras.utils.disable_interactive_logging()

def test_music_genre(new_audio_file):
  # Load individual models
  feature_model1 = load_model('/content/ann_20_mfccs_model.h5')  # Replace with the actual path to your model
  feature_model2 = load_model('/content/cnn_20_mfccs_model.h5')  # Replace with the actual path to your model

  spectrogram_model1 = load_model('/content/CNN_model_MGC_1.h5')  # Replace with the actual path to your model
  spectrogram_model2 = load_model('/content/GRU_model_MGC_1.h5')  # Replace with the actual path to your model
  spectrogram_model3 = load_model('/content/LSTM_model_MGC_1.h5')
  # Define a list of corrected feature models
  feature_models = [feature_model1, feature_model2]

  # Define a list of corrected spectrogram models
  spectrogram_models = [spectrogram_model1, spectrogram_model2, spectrogram_model3]


  # Define the function to extract features from audio
  def extract_features_segment(audio_data, sr):
      chroma_stft_mean = np.mean(librosa.feature.chroma_stft(y=audio_data, sr=sr))
      chroma_stft_var = np.var(librosa.feature.chroma_stft(y=audio_data, sr=sr))
      rms_mean = np.mean(librosa.feature.rms(y=audio_data))
      rms_var = np.var(librosa.feature.rms(y=audio_data))
      spectral_centroid_mean = np.mean(librosa.feature.spectral_centroid(y=audio_data, sr=sr))
      spectral_centroid_var = np.var(librosa.feature.spectral_centroid(y=audio_data, sr=sr))
      spectral_bandwidth_mean = np.mean(librosa.feature.spectral_bandwidth(y=audio_data, sr=sr))
      spectral_bandwidth_var = np.var(librosa.feature.spectral_bandwidth(y=audio_data, sr=sr))
      rolloff_mean = np.mean(librosa.feature.spectral_rolloff(y=audio_data, sr=sr))
      rolloff_var = np.var(librosa.feature.spectral_rolloff(y=audio_data, sr=sr))
      zero_crossing_rate_mean = np.mean(librosa.feature.zero_crossing_rate(audio_data))
      zero_crossing_rate_var = np.var(librosa.feature.zero_crossing_rate(audio_data))
      harmony_mean = np.mean(librosa.effects.harmonic(y=audio_data))
      harmony_var = np.var(librosa.effects.harmonic(y=audio_data))
      perceptr_mean = np.mean(librosa.effects.percussive(y=audio_data))
      perceptr_var = np.var(librosa.effects.percussive(y=audio_data))
      tempo, _ = librosa.beat.beat_track(y=audio_data, sr=sr)
      mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=20)
      mfcc_means = np.mean(mfccs, axis=1)
      mfcc_vars = np.var(mfccs, axis=1)

      # Combine the extracted features into a single array
      return [chroma_stft_mean, chroma_stft_var, rms_mean, rms_var, spectral_centroid_mean,
              spectral_centroid_var, spectral_bandwidth_mean, spectral_bandwidth_var, rolloff_mean,
              rolloff_var, zero_crossing_rate_mean, zero_crossing_rate_var, harmony_mean, harmony_var,
              perceptr_mean, perceptr_var, tempo] + list(mfcc_means) + list(mfcc_vars)

  # Define the function to create spectrogram with temporary directory
  def create_spectrogram(audio_data, sr, temp_dir, input_shape):
      plt.figure(figsize=(3, 3))
      D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_data)), ref=np.max)
      librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log', cmap='viridis')
      plt.axis('off')

      # Use the provided temporary directory
      spectrogram_filename = os.path.join(temp_dir, 'spectrogram.png')
      plt.savefig(spectrogram_filename, bbox_inches='tight', pad_inches=0, transparent=True)
      plt.close()

      return spectrogram_filename

  # Function to load the new audio file and divide it into segments
  def process_audio_file(new_audio_file, segment_duration=3):
      new_audio_data, sr = librosa.load(new_audio_file)

      # Calculate the number of segments
      total_duration = librosa.get_duration(y=new_audio_data, sr=sr)
      num_segments = int(np.ceil(total_duration / segment_duration))

      # Initialize arrays to store predictions
      all_feature_predictions = []
      all_spectrogram_predictions = []
      all_ensemble_predictions = []

      # Accumulate predictions for each segment
      accumulated_predictions = {
          'feature_model1': [],
          'feature_model2': [],
          'spectrogram_model1': [],
          'spectrogram_model2': [],
          'spectrogram_model3': [],
          'ensemble_model': [],
      }

      # Use tempfile to create a unique temporary directory
      with tempfile.TemporaryDirectory() as temp_dir:
          for i in range(num_segments):
              # Extract the audio segment
              start_time = i * segment_duration
              end_time = min((i + 1) * segment_duration, total_duration)
              audio_segment = new_audio_data[int(start_time * sr):int(end_time * sr)]

              # Extract features from the audio segment for feature models
              features_feature_segment = extract_features_segment(audio_segment, sr)

              # Check the type of feature model and reshape accordingly
              if feature_model1 in feature_models:
                  features_feature_segment_model1 = scaler.transform(np.array([features_feature_segment]))
                  features_feature_segment_model1 = features_feature_segment_model1.reshape(1, features_feature_segment_model1.shape[1], )
                  #print('features_feature_segment_model1 shape', features_feature_segment_model1.shape)
              if feature_model2 in feature_models:
                  features_feature_segment_model2 = scaler.transform(np.array([features_feature_segment]))
                  features_feature_segment_model2 = features_feature_segment_model2.reshape(1, features_feature_segment_model2.shape[1], 1)
                  #print('features_feature_segment_model2 shape', features_feature_segment_model2.shape)

              # Predict using feature models
              feature_predictions = []
              if feature_model1 in feature_models:
                  individual_prediction_1 = feature_model1.predict(features_feature_segment_model1)
                  feature_predictions.append(individual_prediction_1)
                  accumulated_predictions['feature_model1'].append(label_encoder.inverse_transform(np.argmax(individual_prediction_1, axis=1)))
              if feature_model2 in feature_models:
                  individual_prediction_2 = feature_model2.predict(features_feature_segment_model2)
                  feature_predictions.append(individual_prediction_2)
                  accumulated_predictions['feature_model2'].append(label_encoder.inverse_transform(np.argmax(individual_prediction_2, axis=1)))
              ensemble_feature = np.mean(feature_predictions, axis=0)

              # Create a spectrogram for the audio segment for spectrogram models
              spectrogram_filename = create_spectrogram(audio_segment, sr, temp_dir, input_shape=(64, 64, 3))

              # Load and preprocess the spectrogram for spectrogram models
              spectrogram_segment = tf.keras.preprocessing.image.load_img(spectrogram_filename,
                                                                        target_size=(input_shape[0], input_shape[1]))
              spectrogram_segment = tf.keras.preprocessing.image.img_to_array(spectrogram_segment)
              spectrogram_segment /= 255.0
              spectrogram_segment = np.expand_dims(spectrogram_segment, axis=0)

              # Print the shape of the spectrogram data
              # print('Spectrogram segment shape:', spectrogram_segment.shape)

              # Predict using spectrogram models
              spectrogram_predictions = []
              for j, spectrogram_model in enumerate(spectrogram_models):
                  individual_prediction = spectrogram_model.predict(spectrogram_segment)
                  spectrogram_predictions.append(individual_prediction)
                  accumulated_predictions[f'spectrogram_model{j+1}'].append(label_encoder.inverse_transform(np.argmax(individual_prediction, axis=1)))
              ensemble_spectrogram = np.mean(spectrogram_predictions, axis=0)

              # Combine the results from feature and spectrogram models
              ensemble_prediction = label_encoder.inverse_transform(np.argmax(ensemble_feature + ensemble_spectrogram, axis=1))

              # Store predictions for each segment
              all_feature_predictions.append(ensemble_feature)
              all_spectrogram_predictions.append(ensemble_spectrogram)
              all_ensemble_predictions.append(ensemble_prediction)
              accumulated_predictions['ensemble_model'].append(ensemble_prediction)


      # Print predictions for each segment
      for i in range(num_segments):
          print(f'\nSegment {i + 1} Predictions:')
          print(f'Feature Model 1 Prediction: {accumulated_predictions["feature_model1"][i][0]}')
          print(f'Feature Model 2 Prediction: {accumulated_predictions["feature_model2"][i][0]}')
          print(f'Spectrogram Model 1 Prediction: {accumulated_predictions["spectrogram_model1"][i][0]}')
          print(f'Spectrogram Model 2 Prediction: {accumulated_predictions["spectrogram_model2"][i][0]}')
          print(f'Spectrogram Model 3 Prediction: {accumulated_predictions["spectrogram_model3"][i][0]}')
          print(f'Ensemble Prediction: {accumulated_predictions["ensemble_model"][i][0]}')
          print()


      # Print most common predictions for each model
      for model_name, predictions in accumulated_predictions.items():
          predictions_labels = [prediction[0] for prediction in predictions]
          most_common_prediction = Counter(predictions_labels).most_common(1)[0][0]
          print(f'Most Common {model_name} Prediction: {most_common_prediction}')


      return all_feature_predictions, all_spectrogram_predictions, all_ensemble_predictions
  all_feature_predictions, all_spectrogram_predictions, all_ensemble_predictions = process_audio_file(new_audio_file)



In [25]:
# blues
new_audio_file = ("/content/drive/MyDrive/music genre classification/Data/testing audio/blues/03. Sweet Home Chicago (Remastered).mp3")

test_music_genre(new_audio_file)


Segment 1 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: blues
Spectrogram Model 2 Prediction: blues
Spectrogram Model 3 Prediction: blues
Ensemble Prediction: blues


Segment 2 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: blues
Spectrogram Model 2 Prediction: blues
Spectrogram Model 3 Prediction: jazz
Ensemble Prediction: blues


Segment 3 Predictions:
Feature Model 1 Prediction: jazz
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: jazz
Spectrogram Model 2 Prediction: jazz
Spectrogram Model 3 Prediction: jazz
Ensemble Prediction: jazz


Segment 4 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: blues
Spectrogram Model 2 Prediction: blues
Spectrogram Model 3 Prediction: jazz
Ensemble Prediction: blues


Segment 5 Predictions:
Feature Model 1 Prediction: country

In [ ]:
# classical
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/ensemble_testing_audios/classical/10 Waltz.wav')

test_music_genre(new_audio_file)

/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(



Segment 1 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: classical
Spectrogram Model 2 Prediction: classical
Spectrogram Model 3 Prediction: classical
Ensemble Prediction: classical


Segment 2 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: classical
Spectrogram Model 2 Prediction: classical
Spectrogram Model 3 Prediction: classical
Ensemble Prediction: classical


Segment 3 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: classical
Spectrogram Model 2 Prediction: classical
Spectrogram Model 3 Prediction: classical
Ensemble Prediction: classical


Segment 4 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: classical
Spectrogram Model 2 Prediction: classical
Spectrogram Model 3 Prediction: classical


In [ ]:
# country
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/ensemble_testing_audios/country/05 - Johnny Cash - Guess things happen that way (1958).FLAC')

test_music_genre(new_audio_file)

/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(



Segment 1 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 2 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 3 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 4 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 5 Predi

In [ ]:
# disco
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/disco/01. Sister Sledge - Hes the Greatest Dancer.mp3')

test_music_genre(new_audio_file)
print('==========================11111111111111111================================')
print('\n========================================================================')

new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/disco/04. The Spinners - Ill Be Around.mp3')

test_music_genre(new_audio_file)
print('=================222222222222222222222222222222=============================')
print('\n========================================================================')

new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/disco/08. Shalamar - A Night To Remember.mp3')

test_music_genre(new_audio_file)
print('==================333333333333333333333333===============================')
print('\n========================================================================')

new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/disco/12. Shirley & Company - Shame, Shame, Shame.mp3')

test_music_genre(new_audio_file)
print('======================44444444444444======================================')
print('\n========================================================================')

new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/disco/16. The Whispers - And the Beat Goes On.mp3')

test_music_genre(new_audio_file)
print('=========================555555555==================================')
print('\n========================================================================')

new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/disco/19. Randy Crawford - You Might Need Somebody.mp3')

test_music_genre(new_audio_file)
print('===========================================================================')
print('\n========================================================================')


/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(



Segment 1 Predictions:
Feature Model 1 Prediction: hiphop
Feature Model 2 Prediction: reggae
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: disco
Spectrogram Model 3 Prediction: hiphop
Ensemble Prediction: hiphop


Segment 2 Predictions:
Feature Model 1 Prediction: metal
Feature Model 2 Prediction: disco
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: pop
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: reggae


Segment 3 Predictions:
Feature Model 1 Prediction: metal
Feature Model 2 Prediction: metal
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: pop
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 4 Predictions:
Feature Model 1 Prediction: hiphop
Feature Model 2 Prediction: metal
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: disco
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 5 Predictions:
Feature Model 1 Prediction: metal
Fe

/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(



Segment 1 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: blues
Ensemble Prediction: blues


Segment 2 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: rock
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: disco
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 3 Predictions:
Feature Model 1 Prediction: pop
Feature Model 2 Prediction: rock
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 4 Predictions:
Feature Model 1 Prediction: pop
Feature Model 2 Prediction: hiphop
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 5 Predictions:
Feature Model 1 Prediction: rock
Featur

In [14]:
# hiphop
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/ensemble_testing_audios/hiphop/06. Mama Said Knock You Out.mp3')

test_music_genre(new_audio_file)

/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(



Segment 1 Predictions:
Feature Model 1 Prediction: hiphop
Feature Model 2 Prediction: hiphop
Spectrogram Model 1 Prediction: hiphop
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: hiphop
Ensemble Prediction: hiphop


Segment 2 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: reggae
Spectrogram Model 1 Prediction: hiphop
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: hiphop
Ensemble Prediction: reggae


Segment 3 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: reggae
Spectrogram Model 1 Prediction: jazz
Spectrogram Model 2 Prediction: jazz
Spectrogram Model 3 Prediction: jazz
Ensemble Prediction: jazz


Segment 4 Predictions:
Feature Model 1 Prediction: hiphop
Feature Model 2 Prediction: hiphop
Spectrogram Model 1 Prediction: hiphop
Spectrogram Model 2 Prediction: hiphop
Spectrogram Model 3 Prediction: hiphop
Ensemble Prediction: hiphop


Segment 5 Predictions:
Feature Model 1 Predicti

In [ ]:
# jazz
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/ensemble_testing_audios/jazz/Miles Davis - So What (Official Audio) [TubeRipper.com].wav')

test_music_genre(new_audio_file)


Segment 1 Predictions:
Feature Model 1 Prediction: jazz
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: jazz
Spectrogram Model 2 Prediction: jazz
Spectrogram Model 3 Prediction: jazz
Ensemble Prediction: jazz


Segment 2 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: jazz
Spectrogram Model 2 Prediction: jazz
Spectrogram Model 3 Prediction: jazz
Ensemble Prediction: classical


Segment 3 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: jazz
Spectrogram Model 2 Prediction: classical
Spectrogram Model 3 Prediction: classical
Ensemble Prediction: classical


Segment 4 Predictions:
Feature Model 1 Prediction: jazz
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: jazz
Spectrogram Model 2 Prediction: jazz
Spectrogram Model 3 Prediction: jazz
Ensemble Prediction: jazz


Segment 5 Predictions:
Feature Model 1 Predicti

In [ ]:
# metal
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/metal/002. Iron Maiden - Run to the Hills (2015 Remaster).mp3')

test_music_genre(new_audio_file)


new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/metal/004. Avenged Sevenfold - Hail to the King.mp3')

test_music_genre(new_audio_file)

new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/metal/006. Machine Head - Locust.mp3')

test_music_genre(new_audio_file)

new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/metal/008. Pantera - Mouth for War.mp3')

test_music_genre(new_audio_file)


Segment 1 Predictions:
Feature Model 1 Prediction: hiphop
Feature Model 2 Prediction: hiphop
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: metal
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: hiphop


Segment 2 Predictions:
Feature Model 1 Prediction: hiphop
Feature Model 2 Prediction: disco
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: disco
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 3 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: disco
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: metal
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 4 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: metal
Spectrogram Model 1 Prediction: metal
Spectrogram Model 2 Prediction: metal
Spectrogram Model 3 Prediction: metal
Ensemble Prediction: metal


Segment 5 Predictions:
Feature Model 1 Prediction: jazz
Fea

/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(



Segment 1 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: pop
Spectrogram Model 2 Prediction: rock
Spectrogram Model 3 Prediction: country
Ensemble Prediction: rock


Segment 2 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: pop
Spectrogram Model 1 Prediction: rock
Spectrogram Model 2 Prediction: rock
Spectrogram Model 3 Prediction: rock
Ensemble Prediction: rock


Segment 3 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: reggae
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 4 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: rock
Spectrogram Model 1 Prediction: rock
Spectrogram Model 2 Prediction: rock
Spectrogram Model 3 Prediction: country
Ensemble Prediction: rock


Segment 5 Predictions:
Feature Model 1 Prediction: rock
Feature Model

In [15]:
# pop
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/pop/Havana (Audio) ft. Young Thug (256 kbps).mp3')

test_music_genre(new_audio_file)


Segment 1 Predictions:
Feature Model 1 Prediction: jazz
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: blues
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: blues
Ensemble Prediction: blues


Segment 2 Predictions:
Feature Model 1 Prediction: jazz
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 3 Predictions:
Feature Model 1 Prediction: jazz
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 4 Predictions:
Feature Model 1 Prediction: pop
Feature Model 2 Prediction: pop
Spectrogram Model 1 Prediction: blues
Spectrogram Model 2 Prediction: blues
Spectrogram Model 3 Prediction: country
Ensemble Prediction: blues


Segment 5 Predictions:
Feature Model 1 Prediction: 

In [ ]:
# reggae
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/ensemble_testing_audios/reggae/Buffalo Soldier (1983) - Bob Marley The Wailers [TubeRipper.com].wav')

test_music_genre(new_audio_file)


Segment 1 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: hiphop
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: reggae
Ensemble Prediction: reggae


Segment 2 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: reggae
Ensemble Prediction: reggae


Segment 3 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: rock
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: reggae
Ensemble Prediction: reggae


Segment 4 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: reggae
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: reggae
Spectrogram Model 3 Prediction: reggae
Ensemble Prediction: reggae


Segment 5 Predictions:
Feature Model 1 Pre

In [16]:
# rock
new_audio_file = ('/content/drive/MyDrive/music genre classification/Data/testing audio/rock/001. Eagles - Hotel California (2013 Remaster).mp3')

test_music_genre(new_audio_file)


Segment 1 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 2 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 3 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 4 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 5 Predictions:
Featu

In [ ]:
pip install youtube-dl pytube


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install ffmpeg


  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6080 sha256=59fa6405cc691f7ad00a2b0c3f258b05a6c937bea2b664809dc99591213193f7
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
from pytube import YouTube
import os
import subprocess

def download_music(url, output_dir, format='wav'):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_file_path = audio_stream.download(output_dir)

    # Convert the downloaded audio to the desired format using ffmpeg command line tool
    output_file_path = os.path.splitext(audio_file_path)[0] + '.' + format
    subprocess.run(['ffmpeg', '-i', audio_file_path, output_file_path])

    # Remove the original downloaded file
    os.remove(audio_file_path)

    return output_file_path



# Example usage:
url = "https://www.youtube.com/watch?v=71Gt46aX9Z4"  # Replace with the actual URL of the music video
output_dir = "/content/drive/MyDrive/music genre classification/Data/testing audio/pop"
audio_file_path = download_music(url, output_dir, format='wav')
print("Downloaded audio file:", audio_file_path)


Downloaded audio file: /content/drive/MyDrive/music genre classification/Data/testing audio/pop/Gary BB Coleman - The Sky is Crying.wav


In [ ]:
from pytube import YouTube
import os
import subprocess
import tempfile

def test_music_using_url(url, format='mp3'):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()

    # Create a temporary directory to store the downloaded file
    with tempfile.TemporaryDirectory() as temp_dir:
        audio_file_path = audio_stream.download(temp_dir)

        # Convert the downloaded audio to the desired format using ffmpeg command line tool
        output_file_path = os.path.join(temp_dir, f'audio.{format}')
        subprocess.run(['ffmpeg', '-i', audio_file_path, output_file_path])

        # Call the test_music_genre function with the temporary audio file
        test_music_genre(output_file_path)


# Example usage:
url = "https://www.youtube.com/watch?v=71Gt46aX9Z4"  # Replace with the actual URL of the music video
test_music_using_url(url, format='mp3')


/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/usr/local/lib/python3.10/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(



Segment 1 Predictions:
Feature Model 1 Prediction: rock
Feature Model 2 Prediction: hiphop
Spectrogram Model 1 Prediction: jazz
Spectrogram Model 2 Prediction: jazz
Spectrogram Model 3 Prediction: pop
Ensemble Prediction: pop


Segment 2 Predictions:
Feature Model 1 Prediction: country
Feature Model 2 Prediction: country
Spectrogram Model 1 Prediction: blues
Spectrogram Model 2 Prediction: blues
Spectrogram Model 3 Prediction: blues
Ensemble Prediction: country


Segment 3 Predictions:
Feature Model 1 Prediction: pop
Feature Model 2 Prediction: jazz
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: hiphop
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 4 Predictions:
Feature Model 1 Prediction: pop
Feature Model 2 Prediction: rock
Spectrogram Model 1 Prediction: country
Spectrogram Model 2 Prediction: country
Spectrogram Model 3 Prediction: country
Ensemble Prediction: country


Segment 5 Predictions:
Feature Model 1 Prediction: pop

In [ ]:
import os
import subprocess
import tempfile
from pytube import YouTube

def download_music_temporary(url, format='mp3', bitrate='128k'):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()

    # Create a temporary directory to store the downloaded file
    with tempfile.TemporaryDirectory() as temp_dir:
        audio_file_path = audio_stream.download(temp_dir)

        # Convert the downloaded audio to the desired format using ffmpeg command line tool
        output_file_path = os.path.join(temp_dir, f'audio.{format}')
        subprocess.run(['ffmpeg', '-i', audio_file_path, '-b:a', bitrate, output_file_path])

        # Call the test_music_genre function with the temporary audio file
        test_music_genre(output_file_path)

In [ ]:
url = "https://www.youtube.com/watch?v=euJ22UqLD5Y"  # Replace with the actual URL of the music video
test_music_using_url(url, format='mp3')


Segment 1 Predictions:
Feature Model 1 Prediction: hiphop
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: disco
Spectrogram Model 3 Prediction: hiphop
Ensemble Prediction: hiphop


Segment 2 Predictions:
Feature Model 1 Prediction: classical
Feature Model 2 Prediction: classical
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: pop
Spectrogram Model 3 Prediction: hiphop
Ensemble Prediction: classical


Segment 3 Predictions:
Feature Model 1 Prediction: reggae
Feature Model 2 Prediction: reggae
Spectrogram Model 1 Prediction: reggae
Spectrogram Model 2 Prediction: disco
Spectrogram Model 3 Prediction: reggae
Ensemble Prediction: reggae


Segment 4 Predictions:
Feature Model 1 Prediction: disco
Feature Model 2 Prediction: disco
Spectrogram Model 1 Prediction: disco
Spectrogram Model 2 Prediction: disco
Spectrogram Model 3 Prediction: disco
Ensemble Prediction: disco


Segment 5 Predictions:
Feature Model 1